# Veritree
https://www.veritree.com

In [1]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [2]:
with open('../input/Verritree/verritree1.json', 'r') as file:
    data1 = json.load(file)
with open('../input/Verritree/verritree2.json', 'r') as file:
    data2 = json.load(file)
with open('../input/Verritree/verritree3.json', 'r') as file:
    data3 = json.load(file)

data = data1 + data2 + data3

In [3]:
if isinstance(data, list) and isinstance(data[0], dict):
    df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 82 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  122 non-null    int64  
 1   public_id                           122 non-null    object 
 2   planting_site_id                    122 non-null    int64  
 3   country_id                          122 non-null    int64  
 4   organization_id                     122 non-null    int64  
 5   subsite_type_id                     122 non-null    int64  
 6   plus_code                           122 non-null    object 
 7   name                                122 non-null    object 
 8   description                         122 non-null    object 
 9   legacy                              122 non-null    int64  
 10  created_at                          122 non-null    object 
 11  updated_at                          122 non-n

In [4]:
columns_rename_mapping = {
    'planting_site_id': 'site_id_reported',
    'id': 'project_id_reported',
    'geofence.coordinates': 'geometry',
    'description': 'project_description_reported',
    'planting_site.country.name': 'country',
    'forest_type.average_survival_rate': 'survival_rate_reported',
    'species': 'species_planted_reported'
}
df = df[list(columns_rename_mapping.keys())]
df.rename(columns=columns_rename_mapping, inplace=True)

In [5]:

unique_project_ids_count = df['project_id_reported'].nunique()


print(f"Number of unique project IDs: {unique_project_ids_count}")

Number of unique project IDs: 122


In [ ]:
df.head(1)

### Fix geometries

In [ ]:
df['geometry'] = df['geometry'].apply(lambda coords: Polygon([coord for sublist1 in coords for sublist2 in sublist1 for coord in sublist2]))

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs = 'EPSG:4326')
gdf['geometry'] = gdf['geometry'].make_valid()

In [ ]:
gdf.head(1)

### Harmonize nomenclature

In [ ]:
gdf["host_name"] = 'Veritree'
gdf["url"] = 'https://www.veritree.com'
gdf["site_sqkm"] = gdf["geometry"].to_crs('EPSG:3857').area / 1e6
gdf['species_planted_reported'] = gdf['species_planted_reported'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
gdf = gdf.assign(species_count_reported=None,trees_planted_reported=None,planting_date_reported=None)
gdf.info()

### Save it

In [ ]:
gdf.to_file('../midsave/veritree.gpkg')